In [1]:
import pandas as pd
from collections import Counter
from PIL import Image,ImageDraw,ImageFont
from scipy.stats import spearmanr
import os
import numpy as np

In [2]:
limefeat_alphasort = pd.read_csv("limefeat_alphasort.csv")
officer_table = pd.read_csv("officer_table.csv")
df = pd.read_csv("lime-robustness-all-score-outcome.csv")

In [3]:
def relister(liststring):
    tmp = liststring.lstrip("[").rstrip("]")
    tmp = tmp.split(",")
    tmp = [item.translate(None," '") for item in tmp]
    return tmp

In [4]:
def setstring2list(setstring):
    tmp = setstring.lstrip("{").rstrip("}")
    tmp = tmp.split(",")
    tmp = [item.translate(None," '") for item in tmp]
    return tmp

In [5]:
officer_table['feats'] = officer_table.feats.apply(relister)

In [6]:
df['unq'] = df.unq.apply(setstring2list)

#### Feature categories

In [7]:
cathue = {
    'ts':'0',
    'fi':'32',
    'ir':'60',
    'shifts':'115',
    'dispatch':'180',
    'arrests':'240',
    'ic':'270',
    'demarrests':'300',
    'ocnd':'200'
    }

catsat = {
    'ts':'100%',
    'fi':'100%',
    'ir':'100%',
    'shifts':'100%',
    'dispatch':'100%',
    'arrests':'100%',
    'ic':'100%',
    'demarrests':'100%',
    'ocnd':'100%'
    }

catval = {
    '1d':'5%',
    '1w':'25%',
    '1m':'50%',
    '1y':'75%',
    '5y':'100%',
    'all':'100%',
    }

#### LIME 10-runs

In [8]:
lime_10runs = df[['officer_id','nsmp','nft','unq','unit_score']]

In [9]:
lime_10runs = lime_10runs.sort_values(by=['unit_score','officer_id','nft'],ascending=[False,True,True])
lime_10runs.reset_index(drop=True,inplace=True)

In [10]:
rectwidth = 10
rectheight = 10
px_w = len(limefeat_alphasort) * rectwidth
px_h = len(lime_10runs) * rectheight

In [11]:
def offtableart(lime_10runs,limefeat_alphasort):
    
    canvas = Image.new('RGB',(px_w,px_h),'hsl(0,0%,100%)')
    officer_id = None
    nft = None
    
    for i in lime_10runs.index:
        feats = lime_10runs.unq.loc[i]
        ycoord = i * rectheight
        
        newid = lime_10runs.officer_id.loc[i]
        newft = lime_10runs.nft.loc[i]  
        
        for feat in feats:
            tmp = limefeat_alphasort[limefeat_alphasort.feature==feat]
            record_type = tmp.record_type.iloc[0]
            idx = tmp.index[0]
            
            xcoord = idx * rectwidth
            bbox = [xcoord, ycoord, xcoord + rectwidth, ycoord + rectheight]
            
            try:
                hue = cathue[record_type]
                sat = catsat[record_type]
                val = '50%'
            except:
                hue = '0'
                sat = '0%'
                val = '100%'

            fillcolor = 'hsl('+hue+","+sat+","+val+")"
            draw = ImageDraw.Draw(canvas)
            draw.rectangle(bbox,fill=fillcolor,outline=None)
        
        if newid!=officer_id:
            draw.line([(0,ycoord),(px_w,ycoord)],width=3,fill='black')
            officer_id = newid
            nft = newft
        elif newft!=nft:
            draw.line([(0,ycoord),(px_w,ycoord)],width=2,fill='black')
            nft = newft
        else:
            draw.line([(0,ycoord),(px_w,ycoord)],width=1,fill='black')
    
    return canvas

In [12]:
offtableart(lime_10runs,limefeat_alphasort).save("offtableart.png")